In [31]:
def Pivoting(A, b, s, n, k):
# A: Coef. of matrix A; 2-D array
# b: Coef. of vector b; 1-D array it is None if i didn't pass one
# n: Dimension of the system of equations
# s: n-element array for storing scaling factors

  p = k

  if s is not None:

    # Finding the largest scaled coefficient in column k
    big = abs(A[k][k] / s[k])
    for i in range(k+1,n) :
      dummy = abs(A[i][k] / s[i]) # dummy number for the scaled value
      if dummy > big:
        big = dummy
        p = i # new pivoting row

    # Next: Swap row p and row k if p != k
    if p != k:
      # Swap row p and row k
      for j in range(k,n) :
        dummy = A[p][j]
        A[p][j] = A[k][j]
        A[k][j] = dummy
      if b is not None:
        dummy = b[p]
        b[p] = b[k]
        b[k] = dummy

      dummy = s[p]
      s[p] = s[k]
      s[k] = dummy
  else:
  # Finding the largest coefficient in column k
    big = abs(A[k][k])
    for i in range(k+1,n) :
      dummy = abs(A[i][k] ) # dummy number for the  value
      if dummy > big:
        big = dummy
        p = i # new pivoting row

    # Swap row p and row k if p != k
    if p != k:
      for j in range(k,n) :
        dummy = A[p][j]
        A[p][j] = A[k][j]
        A[k][j] = dummy
      if b is not None:
        dummy = b[p]
        b[p] = b[k]
        b[k] = dummy


In [32]:
def forward_elimination(A, b, s, n, tol):
# A: Coef. of matrix A; 2-D array
# b: Coef. of vector b; 1-D array it is None if i didn't pass one
# n: Dimension of the system of equations
# tol: Tolerance; smallest possible scaled pivot allowed
# s: n-element array for storing scaling factors

  if s is not None and b is not None:
    for k  in range(n-1):
      Pivoting(A, b, s, n, k) # Partial Pivoting
      if abs(A[k][k] / s[k]) < tol:  # Check for singularity
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]
        b[i] = b[i] - factor * b[k]

    if abs(A[n-1][n-1]/s[n-1]) < tol: # Check for singularity
      return -1
  elif s is None and b is not None: # no scaling
    for k  in range(n-1):
      Pivoting(A, b, None, n, k) # Partial Pivoting
      if abs(A[k][k]) < tol:
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]
        b[i] = b[i] - factor * b[k]

    if abs(A[n-1][n-1]) < tol: # Check for singularity
      return -1
  elif s is not None and b is None:
    for k  in range(n-1):
      Pivoting(A, None, s, n, k) # Partial Pivoting
      if abs(A[k][k] / s[k]) < tol:  # Check for singularity
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]

    if abs(A[n-1][n-1]/s[n-1]) < tol: # Check for singularity
      return -1
  else: # no scaling, b is None
    for k  in range(n-1):
      Pivoting(A, None, None, n, k) # Partial Pivoting
      if abs(A[k][k]) < tol:
        return -1
      for i in range(k+1,n):
        factor = A[i][k] / A[k][k]
        for j in range(k,n): # Changed range from k+1 to k
          A[i][j] = A[i][j] - factor * A[k][j]

    if abs(A[n-1][n-1]) < tol: # Check for singularity
      return -1
  return 0 # Return 0 for successful elimination

In [33]:
def back_substitution(A, b, n):
  x = [0.0 for _ in range(n)]
  x[n-1] = b[n-1] / A[n-1][n-1]
  for i in range(n-2,-1,-1):
    sum_val = 0.0
    for j in range(i+1,n):
      sum_val = sum_val + A[i][j] * x[j]
    x[i] = (b[i] - sum_val) / A[i][i]
  return x


In [34]:
def gauss_elimination(A, b , n, tol, scalling = False):
# A: Coef. of matrix A; 2-D array
# b: Coef. of vector b; 1-D array it is None if i didn't pass one
# n: Dimension of the system of equations
# tol: Tolerance; smallest possible scaled pivot allowed
# scalling: A boolean showing we want scalling or not
  if scalling:
    s = [] # as an n-element array for storing scaling factors
    for i in range(n):
      s.append(abs(A[i][0]))
      for j in range(1,n) :
        if abs(A[i][j]) > s[i]:
          s[i] = abs(A[i][j])
    if b is not None:
      temp = forward_elimination(A, b, s, n, tol)  # forward elimination
      if temp != -1:             # If not singular
        return back_substitution(A, b, n)    # back substitution
      else:
        return None # Indicate singular matrix
    else: # b is None
      temp = forward_elimination(A, None, s, n, tol)  # forward elimination
      if temp != -1:
        return A # Return the modified A if b is None
      else:
        return None # Indicate singular matrix
  else: # no scalling
      if b is not None:
        temp = forward_elimination(A, b, None, n, tol)  # forward elimination, passing None for s
        if temp != -1:             # If not singular
          return back_substitution(A, b, n)    # back substitution
        else:
          return None # Indicate singular matrix
      else: # b is None
        temp = forward_elimination(A, None, None, n, tol)  # forward elimination
        if temp != -1:
          return A # Return the modified A if b is None
        else:
          return None # Indicate singular matrix


In [35]:

import numpy as np

# Helper function to check if two matrices are approximately equal
def are_matrices_approx_equal(A, B, tol=1e-9):
    return np.allclose(A, B, atol=tol)

# Helper function to check if two vectors are approximately equal
def are_vectors_approx_equal(x, y, tol=1e-9):
    return np.allclose(x, y, atol=tol)

print("Test functions for Gaussian Elimination")


Test functions for Gaussian Elimination


### Testing `Pivoting` function

In [36]:
# Test 1: Basic pivoting with scaling and b
print("--- Pivoting Test 1 (with scaling and b) ---")
A = np.array([[0.0, 3.0, 2.0], [1.0, 2.0, 1.0], [2.0, 1.0, 0.0]], dtype=float)
b = np.array([1.0, 2.0, 3.0], dtype=float)
s = np.array([3.0, 2.0, 2.0], dtype=float) # Max abs value in each row
n = 3
k = 0

print("Input A:\n", A)
print("Input b:\n", b)
print("Input s:\n", s)

A_orig = A.copy()
b_orig = b.copy()
s_orig = s.copy()

Pivoting(A, b, s, n, k)

# Expected result after pivoting (row 2 should move to row 0)
expected_A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 3.0, 2.0]], dtype=float)
expected_b = np.array([3.0, 2.0, 1.0], dtype=float)
expected_s = np.array([2.0, 2.0, 3.0], dtype=float)

print("Actual A after Pivoting:\n", A)
print("Actual b after Pivoting:\n", b)
print("Actual s after Pivoting:\n", s)
print("Expected A:\n", expected_A)
print("Expected b:\n", expected_b)
print("Expected s:\n", expected_s)

assert are_matrices_approx_equal(A, expected_A), "Pivoting Test 1 (A) Failed"
assert are_vectors_approx_equal(b, expected_b), "Pivoting Test 1 (b) Failed"
assert are_vectors_approx_equal(s, expected_s), "Pivoting Test 1 (s) Failed"
print("Pivoting Test 1 (with scaling and b) passed.\n")

# Test 2: Pivoting without scaling, with b
print("--- Pivoting Test 2 (without scaling, with b) ---")
A = np.array([[0.0, 3.0, 2.0], [1.0, 2.0, 1.0], [2.0, 1.0, 0.0]], dtype=float)
b = np.array([1.0, 2.0, 3.0], dtype=float)
s = None
n = 3
k = 0

print("Input A:\n", A)
print("Input b:\n", b)

Pivoting(A, b, s, n, k)

# Expected result after pivoting (row 2 has largest abs(A[i][k]) (2.0) for k=0, so it swaps with row 0)
expected_A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 3.0, 2.0]], dtype=float)
expected_b = np.array([3.0, 2.0, 1.0], dtype=float)

print("Actual A after Pivoting:\n", A)
print("Actual b after Pivoting:\n", b)
print("Expected A:\n", expected_A)
print("Expected b:\n", expected_b)

assert are_matrices_approx_equal(A, expected_A), "Pivoting Test 2 (A) Failed"
assert are_vectors_approx_equal(b, expected_b), "Pivoting Test 2 (b) Failed"
print("Pivoting Test 2 (without scaling, with b) passed.\n")

# Test 3: No pivoting needed (largest element already at A[k][k])
print("--- Pivoting Test 3 (no pivoting needed) ---")
A = np.array([[3.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 3.0, 2.0]], dtype=float)
b = np.array([1.0, 2.0, 3.0], dtype=float)
s = np.array([3.0, 2.0, 3.0], dtype=float)
n = 3
k = 0

print("Input A:\n", A)
print("Input b:\n", b)
print("Input s:\n", s)

A_before = A.copy()
b_before = b.copy()
s_before = s.copy()

Pivoting(A, b, s, n, k)

print("Actual A after Pivoting:\n", A)
print("Actual b after Pivoting:\n", b)
print("Actual s after Pivoting:\n", s)
print("Expected A:\n", A_before)
print("Expected b:\n", b_before)
print("Expected s:\n", s_before)

assert are_matrices_approx_equal(A, A_before), "Pivoting Test 3 (A) Failed"
assert are_vectors_approx_equal(b, b_before), "Pivoting Test 3 (b) Failed"
assert are_vectors_approx_equal(s, s_before), "Pivoting Test 3 (s) Failed"
print("Pivoting Test 3 (no pivoting needed) passed.\n")

# Test 4: Pivoting with scaling but b is None
print("--- Pivoting Test 4 (with scaling, b is None) ---")
A = np.array([[0.0, 3.0, 2.0], [1.0, 2.0, 1.0], [2.0, 1.0, 0.0]], dtype=float)
b = None
s = np.array([3.0, 2.0, 2.0], dtype=float)
n = 3
k = 0

print("Input A:\n", A)
print("Input s:\n", s)

Pivoting(A, b, s, n, k)

expected_A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 3.0, 2.0]], dtype=float)
expected_s = np.array([2.0, 2.0, 3.0], dtype=float)

print("Actual A after Pivoting:\n", A)
print("Actual s after Pivoting:\n", s)
print("Expected A:\n", expected_A)
print("Expected s:\n", expected_s)

assert are_matrices_approx_equal(A, expected_A), "Pivoting Test 4 (A) Failed"
assert b is None, "Pivoting Test 4 (b) Failed"
assert are_vectors_approx_equal(s, expected_s), "Pivoting Test 4 (s) Failed"
print("Pivoting Test 4 (with scaling, b is None) passed.\n")

--- Pivoting Test 1 (with scaling and b) ---
Input A:
 [[0. 3. 2.]
 [1. 2. 1.]
 [2. 1. 0.]]
Input b:
 [1. 2. 3.]
Input s:
 [3. 2. 2.]
Actual A after Pivoting:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Actual b after Pivoting:
 [3. 2. 1.]
Actual s after Pivoting:
 [2. 2. 3.]
Expected A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Expected b:
 [3. 2. 1.]
Expected s:
 [2. 2. 3.]
Pivoting Test 1 (with scaling and b) passed.

--- Pivoting Test 2 (without scaling, with b) ---
Input A:
 [[0. 3. 2.]
 [1. 2. 1.]
 [2. 1. 0.]]
Input b:
 [1. 2. 3.]
Actual A after Pivoting:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Actual b after Pivoting:
 [3. 2. 1.]
Expected A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Expected b:
 [3. 2. 1.]
Pivoting Test 2 (without scaling, with b) passed.

--- Pivoting Test 3 (no pivoting needed) ---
Input A:
 [[3. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Input b:
 [1. 2. 3.]
Input s:
 [3. 2. 3.]
Actual A after Pivoting:
 [[3. 1. 0.]
 [1. 2. 1.]
 [0. 3. 2.]]
Actual b after Pivoting:
 [1. 2. 3.]
Actual s

### Testing `forward_elimination` function

In [43]:
# Test 1: Standard case with scaling and b
print("--- Forward Elimination Test 1 (with scaling and b) ---")
A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
b = np.array([3.0, 4.0, 3.0], dtype=float)
s = np.array([2.0, 2.0, 2.0], dtype=float)
n = 3
tol = 1e-9

print("Input A:\n", A)
print("Input b:\n", b)
print("Input s:\n", s)

result = forward_elimination(A, b, s, n, tol)

expected_A = np.array([[2., 1., 0.], [0., 1.5, 1.], [0., 0., 1.33333333]], dtype=float) # approximately 4/3
expected_b = np.array([3., 2.5, 1.33333333], dtype=float) # approximately 4/3

print("Actual A after elimination:\n", A)
print("Actual b after elimination:\n", b)
print("Expected A:\n", expected_A)
print("Expected b:\n", expected_b)
print("Actual result (0 for success):", result)

assert result == 0, "Forward Elimination Test 1 (return value) Failed"
assert are_matrices_approx_equal(A, expected_A), "Forward Elimination Test 1 (A) Failed"
assert are_vectors_approx_equal(b, expected_b), "Forward Elimination Test 1 (b) Failed"
print("Forward Elimination Test 1 (with scaling and b) passed.\n")

# Test 2: Standard case without scaling, with b
print("--- Forward Elimination Test 2 (without scaling, with b) ---")
A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
b = np.array([3.0, 4.0, 3.0], dtype=float)
s = None
n = 3
tol = 1e-9

print("Input A:\n", A)
print("Input b:\n", b)

result = forward_elimination(A, b, s, n, tol)

expected_A_no_scale = np.array([[2., 1., 0.], [0., 1.5, 1.], [0., 0., 1.33333333]], dtype=float) # approximately 4/3
expected_b_no_scale = np.array([3., 2.5, 1.33333333], dtype=float) # approximately 4/3

print("Actual A after elimination:\n", A)
print("Actual b after elimination:\n", b)
print("Expected A:\n", expected_A_no_scale)
print("Expected b:\n", expected_b_no_scale)
print("Actual result (0 for success):", result)

assert result == 0, "Forward Elimination Test 2 (return value) Failed"
assert are_matrices_approx_equal(A, expected_A_no_scale), "Forward Elimination Test 2 (A) Failed"
assert are_vectors_approx_equal(b, expected_b_no_scale), "Forward Elimination Test 2 (b) Failed"
print("Forward Elimination Test 2 (without scaling, with b) passed.\n")

# Test 3: Singular matrix with scaling and b
print("--- Forward Elimination Test 3 (singular matrix with scaling) ---")
A_singular = np.array([[1.0, 1.0, 1.0], [0.0, 0.0, 2.0], [0.0, 0.0, 3.0]], dtype=float)
b_singular = np.array([1.0, 2.0, 3.0], dtype=float)
s_singular = np.array([1.0, 2.0, 3.0], dtype=float)
n = 3
tol = 1e-9

print("Input A_singular:\n", A_singular)
print("Input b_singular:\n", b_singular)

result_singular = forward_elimination(A_singular, b_singular, s_singular, n, tol)
print("Actual result (-1 for singular):", result_singular)

assert result_singular == -1, "Forward Elimination Test 3 (singular) Failed"
print("Forward Elimination Test 3 (singular matrix with scaling) passed.\n")

# Test 4: Singular matrix without scaling, with b
print("--- Forward Elimination Test 4 (singular matrix without scaling) ---")
A_singular = np.array([[1.0, 1.0, 1.0], [0.0, 0.0, 2.0], [0.0, 0.0, 3.0]], dtype=float)
b_singular = np.array([1.0, 2.0, 3.0], dtype=float)
s_singular = None
n = 3
tol = 1e-9

print("Input A_singular:\n", A_singular)
print("Input b_singular:\n", b_singular)

result_singular = forward_elimination(A_singular, b_singular, s_singular, n, tol)
print("Actual result (-1 for singular):", result_singular)

assert result_singular == -1, "Forward Elimination Test 4 (singular) Failed"
print("Forward Elimination Test 4 (singular matrix without scaling) passed.\n")

# Test 5: Standard case with scaling, b is None
print("--- Forward Elimination Test 5 (with scaling, b is None) ---")
A_no_b = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
s_no_b = np.array([2.0, 2.0, 2.0], dtype=float)
n = 3
tol = 1e-9

print("Input A_no_b:\n", A_no_b)
print("Input s_no_b:\n", s_no_b)

result_no_b = forward_elimination(A_no_b, None, s_no_b, n, tol)

expected_A_no_b = np.array([[2., 1., 0.], [0., 1.5, 1.], [0., 0., 1.33333333]], dtype=float) # approximately 4/3

print("Actual A_no_b after elimination:\n", A_no_b)
print("Expected A_no_b:\n", expected_A_no_b)
print("Actual result (0 for success):", result_no_b)

assert result_no_b == 0, "Forward Elimination Test 5 (return value) Failed"
assert are_matrices_approx_equal(A_no_b, expected_A_no_b), "Forward Elimination Test 5 (A) Failed"
print("Forward Elimination Test 5 (with scaling, b is None) passed.\n")

--- Forward Elimination Test 1 (with scaling and b) ---
Input A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 1. 2.]]
Input b:
 [3. 4. 3.]
Input s:
 [2. 2. 2.]
Actual A after elimination:
 [[2.         1.         0.        ]
 [0.         1.5        1.        ]
 [0.         0.         1.33333333]]
Actual b after elimination:
 [3.         2.5        1.33333333]
Expected A:
 [[2.         1.         0.        ]
 [0.         1.5        1.        ]
 [0.         0.         1.33333333]]
Expected b:
 [3.         2.5        1.33333333]
Actual result (0 for success): 0
Forward Elimination Test 1 (with scaling and b) passed.

--- Forward Elimination Test 2 (without scaling, with b) ---
Input A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 1. 2.]]
Input b:
 [3. 4. 3.]
Actual A after elimination:
 [[2.         1.         0.        ]
 [0.         1.5        1.        ]
 [0.         0.         1.33333333]]
Actual b after elimination:
 [3.         2.5        1.33333333]
Expected A:
 [[2.         1.         0.        ]
 [0.         

### Testing `back_substitution` function

In [44]:
# Test 1: Simple 3x3 system
print("--- Back Substitution Test 1 ---")
A = np.array([[2.0, 1.0, 0.0], [0.0, 1.5, 1.0], [0.0, 0.0, 1.33333333]], dtype=float) # Updated A based on correct forward elimination
b = np.array([3.0, 2.5, 1.33333333], dtype=float) # Updated b based on correct forward elimination
n = 3

print("Input A:\n", A)
print("Input b:\n", b)

x = back_substitution(A, b, n)
expected_x = np.array([1.0, 1.0, 1.0], dtype=float) # Expected solution for this system

print("Actual x:\n", x)
print("Expected x:\n", expected_x)

assert are_vectors_approx_equal(x, expected_x), "Back Substitution Test 1 Failed"
print("Back Substitution Test 1 passed.\n")

# Test 2: Another 2x2 system
print("--- Back Substitution Test 2 ---")
A_2x2 = np.array([[1.0, 2.0], [0.0, 3.0]], dtype=float)
b_2x2 = np.array([7.0, 9.0], dtype=float)
n_2x2 = 2

print("Input A_2x2:\n", A_2x2)
print("Input b_2x2:\n", b_2x2)

x_2x2 = back_substitution(A_2x2, b_2x2, n_2x2)
expected_x_2x2 = np.array([1.0, 3.0], dtype=float)

print("Actual x_2x2:\n", x_2x2)
print("Expected x_2x2:\n", expected_x_2x2)

assert are_vectors_approx_equal(x_2x2, expected_x_2x2), "Back Substitution Test 2 Failed"
print("Back Substitution Test 2 passed.\n")

--- Back Substitution Test 1 ---
Input A:
 [[2.         1.         0.        ]
 [0.         1.5        1.        ]
 [0.         0.         1.33333333]]
Input b:
 [3.         2.5        1.33333333]
Actual x:
 [np.float64(1.0), np.float64(1.0), np.float64(1.0)]
Expected x:
 [1. 1. 1.]
Back Substitution Test 1 passed.

--- Back Substitution Test 2 ---
Input A_2x2:
 [[1. 2.]
 [0. 3.]]
Input b_2x2:
 [7. 9.]
Actual x_2x2:
 [np.float64(1.0), np.float64(3.0)]
Expected x_2x2:
 [1. 3.]
Back Substitution Test 2 passed.



### Testing `gauss_elimination` function

In [45]:
# Test 1: Standard case with scaling and b
print("--- Gauss Elimination Test 1 (with scaling and b) ---")
A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
b = np.array([3.0, 4.0, 3.0], dtype=float)
n = 3
tol = 1e-9

print("Input A:\n", A)
print("Input b:\n", b)

x_scaled = gauss_elimination(A.copy(), b.copy(), n, tol, scalling=True)
expected_x = np.array([1.0, 1.0, 1.0], dtype=float)

print("Actual x_scaled:\n", x_scaled)
print("Expected x:\n", expected_x)

assert x_scaled is not None, "Gauss Elimination Test 1 (return value) Failed"
assert are_vectors_approx_equal(x_scaled, expected_x), "Gauss Elimination Test 1 (scaled) Failed"
print("Gauss Elimination Test 1 (with scaling and b) passed.\n")

# Test 2: Standard case without scaling, with b
print("--- Gauss Elimination Test 2 (without scaling, with b) ---")
A = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
b = np.array([3.0, 4.0, 3.0], dtype=float)
n = 3
tol = 1e-9

print("Input A:\n", A)
print("Input b:\n", b)

x_unscaled = gauss_elimination(A.copy(), b.copy(), n, tol, scalling=False)
expected_x = np.array([1.0, 1.0, 1.0], dtype=float)

print("Actual x_unscaled:\n", x_unscaled)
print("Expected x:\n", expected_x)

assert x_unscaled is not None, "Gauss Elimination Test 2 (return value) Failed"
assert are_vectors_approx_equal(x_unscaled, expected_x), "Gauss Elimination Test 2 (unscaled) Failed"
print("Gauss Elimination Test 2 (without scaling, with b) passed.\n")

# Test 3: Singular matrix with scaling and b
print("--- Gauss Elimination Test 3 (singular matrix with scaling) ---")
A_singular = np.array([[1.0, 1.0, 1.0], [0.0, 0.0, 2.0], [0.0, 0.0, 3.0]], dtype=float)
b_singular = np.array([1.0, 2.0, 3.0], dtype=float)
n = 3
tol = 1e-9

print("Input A_singular:\n", A_singular)
print("Input b_singular:\n", b_singular)

x_singular_scaled = gauss_elimination(A_singular.copy(), b_singular.copy(), n, tol, scalling=True)

print("Actual x_singular_scaled (-1 for singular):", x_singular_scaled)

assert x_singular_scaled is None, "Gauss Elimination Test 3 (singular scaled) Failed"
print("Gauss Elimination Test 3 (singular matrix with scaling) passed.\n")

# Test 4: Singular matrix without scaling, with b
print("--- Gauss Elimination Test 4 (singular matrix without scaling) ---")
A_singular = np.array([[1.0, 1.0, 1.0], [0.0, 0.0, 2.0], [0.0, 0.0, 3.0]], dtype=float)
b_singular = np.array([1.0, 2.0, 3.0], dtype=float)
n = 3
tol = 1e-9

print("Input A_singular:\n", A_singular)
print("Input b_singular:\n", b_singular)

x_singular_unscaled = gauss_elimination(A_singular.copy(), b_singular.copy(), n, tol, scalling=False)

print("Actual x_singular_unscaled (-1 for singular):", x_singular_unscaled)

assert x_singular_unscaled is None, "Gauss Elimination Test 4 (singular unscaled) Failed"
print("Gauss Elimination Test 4 (singular matrix without scaling) passed.\n")

# Test 5: A is modified when b is None and scaling is True
print("--- Gauss Elimination Test 5 (b is None, with scaling) ---")
A_modify_scaled = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
n = 3
tol = 1e-9

print("Input A_modify_scaled:\n", A_modify_scaled)

modified_A_scaled = gauss_elimination(A_modify_scaled.copy(), None, n, tol, scalling=True)
expected_modified_A = np.array([[2., 1., 0.], [0., 1.5, 1.], [0., 0., 1.33333333]], dtype=float) # approximately 4/3

print("Actual modified_A_scaled:\n", modified_A_scaled)
print("Expected modified_A:\n", expected_modified_A)

assert modified_A_scaled is not None, "Gauss Elimination Test 5 (return value) Failed"
assert are_matrices_approx_equal(modified_A_scaled, expected_modified_A), "Gauss Elimination Test 5 (A modified scaled) Failed"
print("Gauss Elimination Test 5 (b is None, with scaling) passed.\n")

# Test 6: A is modified when b is None and scaling is False
print("--- Gauss Elimination Test 6 (b is None, without scaling) ---")
A_modify_unscaled = np.array([[2.0, 1.0, 0.0], [1.0, 2.0, 1.0], [0.0, 1.0, 2.0]], dtype=float)
n = 3
tol = 1e-9

print("Input A_modify_unscaled:\n", A_modify_unscaled)

modified_A_unscaled = gauss_elimination(A_modify_unscaled.copy(), None, n, tol, scalling=False)
expected_modified_A = np.array([[2., 1., 0.], [0., 1.5, 1.], [0., 0., 1.33333333]], dtype=float) # approximately 4/3

print("Actual modified_A_unscaled:\n", modified_A_unscaled)
print("Expected modified_A:\n", expected_modified_A)

assert modified_A_unscaled is not None, "Gauss Elimination Test 6 (return value) Failed"
assert are_matrices_approx_equal(modified_A_unscaled, expected_modified_A), "Gauss Elimination Test 6 (A modified unscaled) Failed"
print("Gauss Elimination Test 6 (b is None, without scaling) passed.\n")

--- Gauss Elimination Test 1 (with scaling and b) ---
Input A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 1. 2.]]
Input b:
 [3. 4. 3.]
Actual x_scaled:
 [np.float64(1.0), np.float64(1.0), np.float64(1.0)]
Expected x:
 [1. 1. 1.]
Gauss Elimination Test 1 (with scaling and b) passed.

--- Gauss Elimination Test 2 (without scaling, with b) ---
Input A:
 [[2. 1. 0.]
 [1. 2. 1.]
 [0. 1. 2.]]
Input b:
 [3. 4. 3.]
Actual x_unscaled:
 [np.float64(1.0), np.float64(1.0), np.float64(1.0)]
Expected x:
 [1. 1. 1.]
Gauss Elimination Test 2 (without scaling, with b) passed.

--- Gauss Elimination Test 3 (singular matrix with scaling) ---
Input A_singular:
 [[1. 1. 1.]
 [0. 0. 2.]
 [0. 0. 3.]]
Input b_singular:
 [1. 2. 3.]
Actual x_singular_scaled (-1 for singular): None
Gauss Elimination Test 3 (singular matrix with scaling) passed.

--- Gauss Elimination Test 4 (singular matrix without scaling) ---
Input A_singular:
 [[1. 1. 1.]
 [0. 0. 2.]
 [0. 0. 3.]]
Input b_singular:
 [1. 2. 3.]
Actual x_singular_unscaled (-

In [46]:
print("All tests passed!")

All tests passed!
